<a href="https://colab.research.google.com/github/aCStandke/ReinforcementLearning/blob/main/ThirdStockEnivornment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Third Stock Trading Environment


  This third stock trading environment is based on Adam King's article as found here:[Creating Bitcoin trading bots don’t lose money](https://medium.com/towards-data-science/creating-bitcoin-trading-bots-that-dont-lose-money-2e7165fb0b29). Similar to the first stock trading environment based on Maxim Lapan's implementation as found in chapter eight of his book [Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition](https://www.amazon.com/Deep-Reinforcement-Learning-Hands-optimization/dp/1838826998), the agent is trading in the environment of the [SPY ETF](https://www.etf.com/SPY?L=1) except in this trading environment the agent is tasked with two discrete actions of not only buying, selling or holding shares but also tasked with determining the amount to buy/sell ranging from 1 to 100 (which will be converted into pecentage form i.e. 1/100=1%, 100/100=100%) based on its trading account/balance [trading account](https://www.investopedia.com/terms/t/tradingaccount.asp#:~:text=A%20trading%20account%20is%20an,margin%20requirements%20set%20by%20FINRA.).  


In [1]:
# ignore warning messages because they are annoying lol
import warnings
warnings.filterwarnings('ignore')

# Installing Necessary Package for Training the Trading Agent

To train the Trading Agent the package [Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/index.html) was used. As stated in the docs: 
> Stable Baselines3 (SB3) is a set of reliable implementations of reinforcement learning algorithms in PyTorch. It is the next major version of Stable Baselines. And steems from the paper [Stable-Baselines3: Reliable Reinforcement Learning Implementations](https://jmlr.org/papers/volume22/20-1364/20-1364.pdf)
The algorithms in this package will make it easier for the research community and industry to replicate, refine, and identify new ideas, and will create good baselines to build projects on top of. We expect these tools will be used as a base around which new ideas can be added, and as a tool for comparing a new approach against existing ones. We also hope that the simplicity of these tools will allow beginners to experiment with a more advanced toolset, without being buried in implementation details.

---
## Proximal Policy Optimization(PPO):

Because in this environment the Agent will be executing continous actions, the Proximal Policy Optimization(PPO) algorithm was chosen. As detailed by the authors [PPO](https://arxiv.org/pdf/1707.06347.pdf)


> We propose a new family of policy gradient methods for reinforcement learning, which alternate between sampling data through interaction with the environment, and optimizing a “surrogate” objective function using stochastic gradient ascent. Whereas standard policy gradient methods perform one gradient update per data sample, we propose a novel objective function that enables multiple epochs of minibatch updates. The new methods, which we call proximal policy optimization (PPO), have some of the benefits of trust region policy optimization (TRPO), but they are much simpler to implement, more general, and have better sample complexity (empirically).


PPO uses the following novel objective function:

$L^{CLIP}(θ)=\hat{E}_t[min(r_{t}(θ)\hat{A}_t,clip(r_{t}(θ), 1-ϵ, 1+ϵ)\hat{A}_t]$

*  $\theta$ is the policy parameter
*  $\hat{E}_t$ denotes the empirical expectation over timesteps
*  $r_{t}$ is the ratio of the probability under the new and old policies, respectively
*  $\hat{A}_t$ is the estimated advantage at time t
*  $\epsilon$ is the clipping hyperparameter, usually 0.1 or 0.2


As detailed by the authors [openAI](https://openai.com/blog/openai-baselines-ppo/#ppo)


> This objective implements a way to do a Trust Region update which is compatible with Stochastic Gradient Descent, and simplifies the algorithm by removing the KL penalty and need to make adaptive updates. In tests, this algorithm has displayed the best performance on continuous control tasks and almost matches ACER’s performance on Atari, despite being far simpler to implement


  









In [5]:
!pip install stable-baselines3[extra]
!pip install empyrical
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 37.4 MB/s 
     |████████████████████████████████| 209 kB 95.1 MB/s 
     |████████████████████████████████| 81 kB 10.7 MB/s 
     |████████████████████████████████| 78 kB 9.2 MB/s 
     |████████████████████████████████| 147 kB 83.1 MB/s 
     |████████████████████████████████| 112 kB 70.6 MB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=9e52eb7f34017bf7b0f08b3a9fb63a1e7b869c8dca107a194a4bd6a57f9c0253
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


# Installing the Necessary Packages for Visualizing the Trading Agent's Envirnoment on Google Colab Notebooks

In [3]:
!pip install mpl_finance #used for plotting the candelstick graph
!pip install moviepy #
!pip install imageio_ffmpeg #
!pip install pyvirtualdisplay > /dev/null 2>&1 #used to create a display for vm
!apt-get install x11-utils > /dev/null 2>&1 #
!pip install pyglet==v1.3.2 > /dev/null 2>&1 #
!apt-get install -y xvfb python-opengl > /dev/null 2>&1 #

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 11.3 MB/s 


In [23]:
import random
import gym
from gym import spaces
from gym.utils import seeding
import pandas as pd
import numpy as np
import json
import datetime as dt
import optuna
from typing import Callable, Dict, List, Optional, Tuple, Type, Union
from stable_baselines3 import PPO
from stable_baselines3.common.utils import constant_fn
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_util import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_checker import VecCheckNan, check_env
from stable_baselines3.common.callbacks import BaseCallback
from empyrical import sortino_ratio, calmar_ratio, omega_ratio
import sqlite3
from sqlite3 import Error
import torch as th
import torch.nn as nn
import collections
import datetime
from sklearn import preprocessing
import math
import os
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

In [7]:
# stock environment parameters
MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 4294967295
LOOKBACK_WINDOW_SIZE = 10
MAX_STEPS = 20000
INITIAL_ACCOUNT_BALANCE = 10000
# default percentage of stock price trading agent pays broker when 
# buying/selling, default is 0.1% (i.e. very reasonable)
DA_COMMISION = 0.1

In [28]:
# Stock/ETF Trading Enviornment
class StockTradingEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, data):
        super(StockTradingEnv, self).__init__()
        self.data = data
        self.scale = preprocessing.MinMaxScaler()
        self.random_ofs_on_reset = True
        self.reward_func = 'calmar'
        self.bars_count = LOOKBACK_WINDOW_SIZE
        self.commission = DA_COMMISION
        self.hold= False

        # Actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float32)

        # Prices contains the OHCL values for the last five prices
        self.observation_space = spaces.Box(
            low=0, high=1, shape=self.shape, dtype=np.float32)
        
        self.seed()

    def reset(self):
      # random offset portion 
      bars = self.bars_count
      if self.random_ofs_on_reset:
        offset = self.np_random.choice(self.data.high.shape[0]-bars*10)+bars
      else:
        offset = bars
      self._reset(offset)
      return self._next_observation()

    def _reset(self, offset):
      self.trades = []
      self.balance = INITIAL_ACCOUNT_BALANCE
      self.netWorth = INITIAL_ACCOUNT_BALANCE
      self.max_net_worth = INITIAL_ACCOUNT_BALANCE
      self.standkeMaxBenchShares = 0
      self.shares_held  = 0
      self._offset = offset
      # setting account history portion
      self.account_history = np.repeat([[self.netWorth/MAX_ACCOUNT_BALANCE]], LOOKBACK_WINDOW_SIZE, axis=1)

    # shape of observation space is 2D
    @property
    def shape(self):
      return (6, self.bars_count)

    def _next_observation(self):
      res = np.zeros(shape=(6, self.bars_count), dtype=np.float32)
      ofs = self.bars_count-1
      res[0] = self.data.volume[self._offset-ofs:self._offset+1]
      res[1] = self.data.high[self._offset-ofs:self._offset+1]
      res[2] = self.data.low[self._offset-ofs:self._offset+1]
      res[3] = self.data.open[self._offset-ofs:self._offset+1]
      res[4] = self.account_history[0][-self.bars_count:]
      res[5] = self.data.close[self._offset-ofs:self._offset+1]
      res = np.float32(res)
      return res
       
    def _take_action(self, action):
      reward = 0
      current_price = self._cur_close()
      action_type = action[0]
      amount = action[1]
      
      shares_bought = 0
      shares_sold = 0
      additional_cost = 0
      sales = 0


      if action_type < 1 :
        # Buy amount % of balance in shares
        total_possible = self.balance / (current_price * (1+self.commission))
        shares_bought = total_possible * amount
        additional_cost = shares_bought * current_price * (1+self.commission)
        self.balance -= additional_cost
        self.standkeMaxBenchShares += shares_bought
        self.shares_held += shares_bought
        
        
        # visualization portion
        if shares_bought > 0:
          self.trades.append({'step': self._offset, 'shares': shares_bought, 
                              'total': additional_cost, 'type': "buy"})
          
          
      elif action_type < 2:
        # Sell amount % of shares held
        shares_sold = self.shares_held * amount  
        sales = shares_sold * current_price * (1 - self.commission)
        self.balance += sales
        self.standkeMaxBenchShares -= shares_sold
        self.shares_held -= shares_sold
        

        # visualization portion
        if shares_sold > 0:
          self.trades.append({'step': self._offset, 'shares': -shares_sold, 
                                  'total': shares_sold * current_price, 'type': "sell"})  
          
      
      self.netWorth = self.balance + self.shares_held * current_price
      
      if self.netWorth > self.max_net_worth:
        self.max_net_worth = self.netWorth

      # updating account history
      self.account_history = np.append(self.account_history, [[self.netWorth/MAX_ACCOUNT_BALANCE]], axis=1)
      # Reward Calculations
      returns = self.account_history[0][-self.bars_count:]
      if self.reward_func == 'BalenceReward':
        delay_modifier = (self._offset / MAX_STEPS)
        reward = self.balance * delay_modifier
      elif self.reward_func == 'sortinoRewardRatio':
        ratio = sortino_ratio(returns, period="daily")
        if ratio < 0: 
          reward = 0
        elif ratio > 0:
          reward = 1
        else:
          reward= ratio
      elif self.reward_func == 'calmarRewardRatio':
        ratio = calmar_ratio(returns, period="daily")
        if ratio < 0: 
          reward = 0
        elif ratio > 0:
          reward = 1
        else:
          reward= ratio
      elif self.reward_func == 'omegaRewardRatio':
        ratio = omega_ratio(returns,  annualization=self.bars_count)
        if ratio < 0: 
          reward = 0
        elif ratio > 0:
          reward = 1
        else:
          reward= ratio
      elif self.reward_func == 'StandkeCurrentValueReward':
        prev_net = returns[-2]
        current_net = returns[-1]
        if (current_net-prev_net)<0:
          reward += 0
        else:
          reward += 1
          if current_net > self.max_net_worth:
            reward *= 10
      elif self.reward_func == 'StandkeSmallDrawDownReward':
        mx = np.max(returns)
        mi = np.min(returns)
        ratio = round(abs(mx-mi/mx), 1) 
        if math.isclose(ratio, 0.0):
          reward = 11  
        elif math.isclose(ratio, 0.1):
          reward = 1/0.1 
        elif math.isclose(ratio, 0.2):
          reward = 1/0.2
        elif math.isclose(ratio, 0.3):
          reward = 1/0.3
        elif math.isclose(ratio, 0.4):
          reward = 1/0.4
        elif math.isclose(ratio, 0.5):
          reward = 1/0.5
        elif math.isclose(ratio, 0.6):
          reward = 1/0.6
        elif math.isclose(ratio, 0.7):
          reward = 1/0.7
        elif math.isclose(ratio, 0.8):
          reward = 1/0.8
        elif math.isclose(ratio, 0.9):
          reward = 1/0.9
        else:
          reward = 0
      elif self.reward_func == 'StandkeLargeDrawDownReward':
        mx = np.max(returns)
        mi = np.min(returns)
        ratio = round(abs(mx-mi/mx), 1) 
        if math.isclose(ratio, 0.0):
          reward = 0
        elif math.isclose(ratio, 0.1):
          reward = 0.1 
        elif math.isclose(ratio, 0.2):
          reward = 0.2
        elif math.isclose(ratio, 0.3):
          reward = 0.3
        elif math.isclose(ratio, 0.4):
          reward = 0.4
        elif math.isclose(ratio, 0.5):
          reward = 0.5
        elif math.isclose(ratio, 0.6):
          reward = 0.6
        elif math.isclose(ratio, 0.7):
          reward = 0.7
        elif math.isclose(ratio, 0.8):
          reward = 0.8
        elif math.isclose(ratio, 0.9):
          reward = 0.9
        else:
          reward = 1
      elif self.reward_func == 'StandkeSumofDifferenceReward':
        reward = np.sum(np.diff(returns))
      else:
        reward = np.mean(returns)

      return reward if abs(reward) != np.inf and not np.isnan(reward) else 0

      
    def _cur_close(self):
      """
      Calculate real close price for the current bar
      """
      return self.data.real_close[self._offset]

    def step(self, action):
      # Execute one time step within the environment
      reward = self._take_action(action)
    
      self._offset += 1

      if self._offset >= self.data.close.shape[0]-1 or self.netWorth <= 0 or self.netWorth>=MAX_ACCOUNT_BALANCE:
        done=True
      else:
        done=False
  
      obs = self._next_observation()

      info = {"Net Worth":self.netWorth, "reward": reward}
      
      return obs, reward, done, info

    def _render_to_file(self, filename='render.txt'):
      f = open(filename, 'a+')
      f.write(f"Step: {self._offset}\n")
      f.write(f"Date: {self.data.date[self._offset]}\n")
      f.write(f"Net Worth: {self.netWorth}\n")
      f.write(f"Balence: {self.balance}\n")
      f.write(f"StandkeMaxBenchShares: {self.standkeMaxBenchShares}\n")
      f.close()
    

    def render(self, mode='file', title="Agent's Trading Screen", **kwargs):
      # Render the environment to the screen
      if mode == 'file':
        self._render_to_file()


    def seed(self, seed=None):
      self.np_random, seed1 = seeding.np_random(seed)
      seed2 = seeding.hash_seed(seed1+1) % 2**33
      return [seed1, seed2]


In [17]:
# using sklearn's min-max scaler for the relative high and low
x=preprocessing.MinMaxScaler()

# taken from https://machinelearningmastery.com/remove-trends-seasonality-difference-transform-python/
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = np.log(dataset[i]) - np.log(dataset[i - interval])
		diff.append(value)
	return diff
 
# training data
df = pd.read_csv('/content/drive/MyDrive/Datasets/StockMarketData/archive/Data/ETFs/spy.us.txt')
df = df.sort_values('Date')
data=df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

# making OHLC data stationary before calculating relative and normalizing 
diff_o = np.array(difference(data['Open'], 1))
diff_h = np.array(difference(data['High'], 1))
diff_l = np.array(difference(data['Low'], 1))
diff_c = np.array(difference(data['Close'], 1))
# volumne data
vol = data['Volume'].values/MAX_NUM_SHARES
# year data of year-month-day form
dt = data['Date'].array
# calculating relative prices and normalizing data
o =  (diff_o-diff_l)/(diff_h-diff_l)
o =  x.fit_transform(o.reshape(-1,1)).reshape(-1)
rc = (diff_c-diff_l)/(diff_h-diff_l)
rc = x.fit_transform(rc.reshape(-1,1)).reshape(-1)

rh = x.fit_transform(diff_h.reshape(-1,1)).reshape(-1)
rl = x.fit_transform(diff_l.reshape(-1,1)).reshape(-1)

Train_Data = collections.namedtuple('Data', field_names=['date','high', 'low', 'close', 'open', 'volume', 'real_open',  'real_close', 'real_high', 'real_low', 'real_vol'])
train = Train_Data(date=dt,high=rh, low=rl, close=rc, open=o, volume=vol, real_open=data['Open'].values, real_close=data['Close'].values, real_high=data['High'].values, real_low=data['Low'].values, real_vol=data['Volume'].values)

In [18]:
# Testing data
test = pd.read_csv('/content/drive/MyDrive/Datasets/StockMarketData/test.csv')
t_df = test.sort_values('Date')
data_two=t_df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

# making OHLC data stationary before calculating relative and normalizing 
diff_o = np.array(difference(data_two['Open'], 1))
diff_h = np.array(difference(data_two['High'], 1))
diff_l = np.array(difference(data_two['Low'], 1))
diff_c = np.array(difference(data_two['Close'], 1))
# volumne data
vol = data_two['Volume'].values/MAX_NUM_SHARES
# year data of year-month-day form
dt = data_two['Date'].array
# calculating relative prices and normalizing data
o =  (diff_o-diff_l)/(diff_h-diff_l)
o =  x.fit_transform(o.reshape(-1,1)).reshape(-1)
rc = (diff_c-diff_l)/(diff_h-diff_l)
rc = x.fit_transform(rc.reshape(-1,1)).reshape(-1)

rh = x.fit_transform(diff_h.reshape(-1,1)).reshape(-1)
rl = x.fit_transform(diff_l.reshape(-1,1)).reshape(-1)

Test_Data = collections.namedtuple('Data', field_names=['date','high', 'low', 'close', 'open', 'volume', 'real_open', 'real_close', 'real_high', 'real_low', 'real_vol'])
test = Test_Data(date=dt,high=rh, low=rl, close=rc, open=o, volume=vol, real_open=data['Open'].values, real_close=data_two['Close'].values, real_high=data_two['High'].values, real_low=data_two['Low'].values, real_vol=data['Volume'].values)

# Creating the Shared Standke Policy/Value Network Class

In [19]:
class StandkeExtractor(BaseFeaturesExtractor):
  def __init__(self, observation_space=gym.spaces.Box, features_dim=128):
        super(StandkeExtractor, self).__init__(observation_space, features_dim)
        
        input = observation_space.shape[0]

        # Feature Extractor
        self.cnn = nn.Sequential(
            nn.Conv1d(input, 128, kernel_size=2),
            nn.ReLU(),
            nn.Conv1d(128, 512, kernel_size=4),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        
  def forward(self, observations):
    return self.cnn(observations)



class StandkeNetwork(nn.Module):
  def __init__(self,feature_dim=3072, last_layer_dim_pi=64, last_layer_dim_vf=64):
        super(StandkeNetwork, self).__init__()

        # IMPORTANT:
        # Save output dimensions, used to create the distributions
        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf

         # Policy Network
        self.policy_net = nn.Sequential(
            nn.Linear(feature_dim, 256),
            nn.BatchNorm1d(256),
            nn.Tanh(),
            nn.Linear(256,last_layer_dim_pi),
            nn.BatchNorm1d(last_layer_dim_pi), 
            nn.Tanh(),  
        )

         # Value Network
        self.value_net = nn.Sequential(
           nn.Linear(feature_dim, 512),
           nn.BatchNorm1d(512),
           nn.Dropout(),
           nn.Tanh(),
           nn.Linear(512,256),
           nn.BatchNorm1d(256),
           nn.Dropout(),
           nn.Tanh(),
           nn.Linear(256,last_layer_dim_vf),
           nn.BatchNorm1d(last_layer_dim_vf),
           nn.Tanh(),
           )



  def forward_actor(self, features):
        return self.policy_net(features)

  def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)
  

class SharedStandkePolicy(ActorCriticPolicy):
  def __init__(self,observation_space=gym.spaces.Box,action_space=gym.spaces.Box, lr_schedule=constant_fn(0.0003), activation_fn=nn.ReLU,*args,**kwargs):
    
        super(SharedStandkePolicy, self).__init__(observation_space,action_space, lr_schedule, activation_fn,*args,**kwargs)
        self.soft = nn.Softplus()
        # shared features extractor for the actor and the critic
        self.policy_features_extractor = StandkeExtractor(observation_space)
        delattr(self, "features_extractor")  # remove the shared features extractor

        # Do not Disable orthogonal initialization
        self.ortho_init = True

  def _build_mlp_extractor(self):
    self.mlp_extractor = StandkeNetwork()

  def extract_features(self, obs: th.Tensor):
    policy_features = self.policy_features_extractor(obs)
    value_features = self.policy_features_extractor(obs)
    return policy_features, value_features
  
  def forward(self, obs: th.Tensor, deterministic: bool = False): 
    policy_features, value_features = self.extract_features(obs)
    latent_pi = self.mlp_extractor.forward_actor(policy_features)
    latent_vf = self.mlp_extractor.forward_critic(value_features)

    # Evaluate the values for the given observations
    values = self.value_net(latent_vf)
    soft_pi = self.soft((latent_pi+-.2))
    distribution = self._get_action_dist_from_latent(soft_pi)
    actions = distribution.get_actions(deterministic=deterministic)
    log_prob = distribution.log_prob(actions)
    return actions, values, log_prob

  def evaluate_actions(self, obs: th.Tensor, actions: th.Tensor): 
    policy_features, value_features = self.extract_features(obs)
    latent_pi = self.mlp_extractor.forward_actor(policy_features)
    latent_vf = self.mlp_extractor.forward_critic(value_features)
    soft_pi = self.soft((latent_pi+-.2))
    distribution = self._get_action_dist_from_latent(soft_pi)
    log_prob = distribution.log_prob(actions)
    values = self.value_net(latent_vf)
    return values, log_prob, distribution.entropy()

  def get_distribution(self, obs: th.Tensor):
    policy_features, _ = self.extract_features(obs)
    latent_pi = self.mlp_extractor.forward_actor(policy_features)
    return self._get_action_dist_from_latent(latent_pi)

  def predict_values(self, obs: th.Tensor):
    _, value_features = self.extract_features(obs)
    latent_vf = self.mlp_extractor.forward_critic(value_features)
    return self.value_net(latent_vf)


# Creating the Seperate Standke Policy/ValueNetwork Class

In [20]:
class StandkeExtractor(BaseFeaturesExtractor):
  def __init__(self, observation_space=gym.spaces.Box, features_dim=128):
        super(StandkeExtractor, self).__init__(observation_space, features_dim)
        
        input = observation_space.shape[0]

        # Feature Extractor
        self.cnn = nn.Sequential(
            nn.Conv1d(input, 128, kernel_size=2),
            nn.ReLU(),
            nn.Conv1d(128, 512, kernel_size=4),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        
  def forward(self, observations):
    return self.cnn(observations)



class StandkeNetwork(nn.Module):
  def __init__(self,feature_dim=3072, last_layer_dim_pi=64, last_layer_dim_vf=64):
        super(StandkeNetwork, self).__init__()

        # IMPORTANT:
        # Save output dimensions, used to create the distributions
        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf

         # Policy Network
        self.policy_net = nn.Sequential(
            nn.Linear(feature_dim, 256),
            nn.BatchNorm1d(256),
            nn.Tanh(),
            nn.Linear(256,last_layer_dim_pi),
            nn.BatchNorm1d(last_layer_dim_pi), 
            nn.Tanh(),  
        )

         # Value Network
        self.value_net = nn.Sequential(
           nn.Linear(feature_dim, 512),
           nn.BatchNorm1d(512),
           nn.Dropout(),
           nn.Tanh(),
           nn.Linear(512,256),
           nn.BatchNorm1d(256),
           nn.Dropout(),
           nn.Tanh(),
           nn.Linear(256,last_layer_dim_vf),
           nn.BatchNorm1d(last_layer_dim_vf),
           nn.Tanh(),
           )



  def forward_actor(self, features: th.Tensor) -> th.Tensor:
        return self.policy_net(features)

  def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)
  

class StandkePolicy(ActorCriticPolicy):
  def __init__(self,observation_space=gym.spaces.Box,action_space=gym.spaces.Box, lr_schedule=constant_fn(0.0003), activation_fn=nn.ReLU,*args,**kwargs):
    
        super(StandkePolicy, self).__init__(observation_space,action_space, lr_schedule, activation_fn,*args,**kwargs)
        self.soft = nn.Softplus()
        # non-shared features extractors for the actor and the critic
        self.policy_features_extractor = StandkeExtractor(observation_space)
        self.value_features_extractor = StandkeExtractor(observation_space)
        delattr(self, "features_extractor")  # remove the shared features extractor

        # Do not Disable orthogonal initialization
        self.ortho_init = True

  def _build_mlp_extractor(self):
    self.mlp_extractor = StandkeNetwork()

  def extract_features(self, obs: th.Tensor):
    policy_features = self.policy_features_extractor(obs)
    value_features = self.value_features_extractor(obs)
    return policy_features, value_features
  
  def forward(self, obs: th.Tensor, deterministic: bool = False): 
    policy_features, value_features = self.extract_features(obs)
    latent_pi = self.mlp_extractor.forward_actor(policy_features)
    latent_vf = self.mlp_extractor.forward_critic(value_features)

    # Evaluate the values for the given observations
    values = self.value_net(latent_vf)
    soft_pi = self.soft((latent_pi+-.2))
    distribution = self._get_action_dist_from_latent(soft_pi)
    actions = distribution.get_actions(deterministic=deterministic)
    log_prob = distribution.log_prob(actions)
    return actions, values, log_prob

  def evaluate_actions(self, obs: th.Tensor, actions: th.Tensor): 
    policy_features, value_features = self.extract_features(obs)
    latent_pi = self.mlp_extractor.forward_actor(policy_features)
    latent_vf = self.mlp_extractor.forward_critic(value_features)
    soft_pi = self.soft((latent_pi+-.2))
    distribution = self._get_action_dist_from_latent(soft_pi)
    log_prob = distribution.log_prob(actions)
    values = self.value_net(latent_vf)
    return values, log_prob, distribution.entropy()

  def get_distribution(self, obs: th.Tensor):
    policy_features, _ = self.extract_features(obs)
    latent_pi = self.mlp_extractor.forward_actor(policy_features)
    return self._get_action_dist_from_latent(latent_pi)

  def predict_values(self, obs: th.Tensor):
    _, value_features = self.extract_features(obs)
    latent_vf = self.mlp_extractor.forward_critic(value_features)
    return self.value_net(latent_vf)

In [22]:
class TensorboardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(TensorboardCallback, self).__init__(verbose)
        self.mean_reward = []

    def _on_step(self) -> bool:
      self.mean_reward.append(self.locals['infos'][0]['reward'])
      if (self.num_timesteps % 10000 == 0):
        self.logger.record('Net Worth', self.locals['infos'][0]['Net Worth'])
        self.logger.record('Mean Reward', np.mean(self.mean_reward))
        self.mean_reward.clear()
      return True

# Creating SQLite Database to Store Hyperparmaters 

In [27]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"/content/drive/MyDrive/RLmodels/sqlite:params.db")

2.6.0


# HyperParmater Tuning 

In [26]:
envs =  DummyVecEnv([lambda: StockTradingEnv(train)])

def optimize(n_trials = 5000, n_jobs = 4):
    study = optuna.create_study(study_name='optimize_hyperparmaters', storage="/content/drive/MyDrive/RLmodels/sqlite:params.db", load_if_exists=True)
    study.optimize(objective_fn, n_trials=n_trials, n_jobs=n_jobs)

def objective_fn(trial):
    env_params = optimize_envs(trial)
    agent_params = optimize_ppo(trial)
    
    train_env, validation_env = initialize_envs(**env_params)
    model = PPO(StandkePolicy, envs, **agent_params)
    
    model.learn(len(train_env.df))
    
    rewards, done = [], False

    obs = validation_env.reset()
    for i in range(len(validation_env.df)):
        action, _ = model.predict(obs)
        obs, reward, done, _ = validation_env.step(action)
        rewards += reward
    
    return -np.mean(rewards)

def optimize_ppo(trial):
    return {
        'n_steps': int(trial.suggest_loguniform('n_steps', 16, 2048)),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-8, 1e-1),
        'cliprange': trial.suggest_uniform('cliprange', 0.1, 0.4),
        'noptepochs': int(trial.suggest_loguniform('noptepochs', 1, 48)),
        'lam': trial.suggest_uniform('lam', 0.8, 1.)
    }

def optimize_envs(trial):
    return {
        'reward_func': trial.suggest_categorical("reward_func", ['BalenceReward', 'sortinoRewardRatio',
         'calmarRewardRatio', 'omegaRewardRatio', 
         'StandkeCurrentValueReward','StandkeSmallDrawDownReward', 
         'StandkeLargeDrawDownReward','StandkeSumofDifferenceReward', 
         'Mean'])
    }

# run optimization to find good hyperparmaters for agent and enviornmnet
optimize()

In [ ]:
# loading the hyperparmeters for training
study = optuna.load_study(study_name='optimize_hyperparmaters', storage='/content/drive/MyDrive/RLmodels/sqlite:params.db')
params = study.best_trial.params
env_params = {
    'reward_func': params['reward_func'])
}
model_params = {
    'n_steps': int(params['n_steps']),
    'gamma': params['gamma'],
    'learning_rate': params['learning_rate'],
    'ent_coef': params['ent_coef'],
    'cliprange': params['cliprange'],
    'noptepochs': int(params['noptepochs']),
    'lam': params['lam']
}

# Training and Validation Portion

In [ ]:
# number of learning steps to train RL model is set to 200K
MAX_STEPS = 2e5
# the number of parallel environments for training  
ENV = 1
MODEL = "StandkeSharedPV_neg0.2offset"

In [ ]:
# create evaluation env that takes in test data to save best model 
eval_env = DummyVecEnv([lambda: StockTradingEnv(test,**env_params)])
eval_callback = EvalCallback(eval_env, best_model_save_path=f'/content/drive/MyDrive/RLmodels/bestPPO/{MODEL}',
                             log_path='/content/drive/MyDrive/RLmodels/logs/', eval_freq=10000,
                             deterministic=False, render=False)

# create training envs that takes in training data for training
envs =  DummyVecEnv([lambda: StockTradingEnv(train, **env_params) for _ in range(0,ENV)])


# optional additional keyword parameters to pass to model 
policy_kwargs = dict()

'''training model using the Seperate Standke Policy/Value network'''
model = PPO(StandkePolicy, envs, **model_params, verbose=1, tensorboard_log=f"/content/PPO_SPY_tensorboard/{MODEL}", policy_kwargs=policy_kwargs)
check_env(StockTradingEnv(train, random_ofs_on_reset=True))
VecCheckNan(envs, raise_exception=True, check_inf=True)

'''training model using the Shared Standke Policy/Value network''' 
# model = PPO(SharedStandkePolicy, envs, verbose=1, tensorboard_log=f"/content/PPO_SPY_tensorboard/{}", policy_kwargs=policy_kwargs)
# check_env(StockTradingEnv(train, random_ofs_on_reset=True))
# VecCheckNan(envs, raise_exception=True, check_inf=True)


# General explanation of log output 

As detailed by araffin in his commit [Add explanation of logger output](https://github.com/DLR-RM/stable-baselines3/pull/803/files), for a given log block such as

```
-----------------------------------------
  | eval/                   |             |
  |    mean_ep_length       | 200         |
  |    mean_reward          | -157        |
  | rollout/                |             |
  |    ep_len_mean          | 200         |
  |    ep_rew_mean          | -227        |
  | time/                   |             |
  |    fps                  | 972         |
  |    iterations           | 19          |
  |    time_elapsed         | 80          |
  |    total_timesteps      | 77824       |
  | train/                  |             |
  |    approx_kl            | 0.037781604 |
  |    clip_fraction        | 0.243       |
  |    clip_range           | 0.2         |
  |    entropy_loss         | -1.06       |
  |    explained_variance   | 0.999       |
  |    learning_rate        | 0.001       |
  |    loss                 | 0.245       |
  |    n_updates            | 180         |
  |    policy_gradient_loss | -0.00398    |
  |    std                  | 0.205       |
  |    value_loss           | 0.226       |
  -----------------------------------------
```
``eval/`` 
- ``mean_ep_length``: Mean episode length
- ``mean_reward``: Mean episodic reward (during evaluation)
``rollout/``
- ``ep_len_mean``: Mean episode length (averaged over 100 episodes)
- ``ep_rew_mean``: Mean episodic training reward (averaged over 100 episodes)
``time/``
- ``episodes``: Total number of episodes
- ``fps``: Number of frames per seconds (includes time taken by gradient update)
- ``iterations``: Number of iterations (data collection + policy update for A2C/PPO)
- ``time_elapsed``: Time in seconds since the beginning of training
- ``total_timesteps``: Total number of timesteps (steps in the environments)
``train/``
- ``entropy_loss``: Mean value of the entropy loss (negative of the average policy entropy). 
  * ⚠**According to the formula as detailed [model](https://github.com/openai/baselines/blob/ea25b9e8b234e6ee1bca43083f8f3cf974143998/baselines/ppo2/model.py#L91) on line 91, if ent_coef is 0 this term should not matter which is the default hyperparamter setting; difficult to interpret for this env due to it being negative**⚠
  * **Furthermore according to [The 37 Implementation Details of Proximal Policy Optimization](https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/) which cites [Andrychowicz, et al. (2021)](https://openreview.net/forum?id=nIAxjsniDzg) overall find no evidence that the entropy term improves performance on continuous control environments (decision C13, figure 76 and 77)**
- ``clip_fraction``: mean fraction of surrogate loss that was clipped (above clip_range threshold) for PPO.
- ``clip_range``: Current value of the clipping factor for the surrogate loss of PPO
- ``entropy_loss``: Mean value of the entropy loss (negative of the average policy entropy)
    *  want the entropy to be decreasing slowly and smoothly over the course of training, as the agent trades exploration in favor of exploitation.
- ``learning_rate``: Current learning rate value
- ``n_updates``: Number of gradient updates applied so far
- ``policy_gradient_loss``: Current value of the policy gradient loss (its value does not have much meaning)(lol I did not say this 😸)
- ``std``: Current standard deviation of the noise when using generalized State-Dependent Exploration (gSDE) (which by default is not used)

# Important Training Metrics to Focus On!!!! ✅✅✅✅✅✅✅✅✅
- ``approx_kl``: approximate mean KL divergence between old and new policy (for PPO), it is an estimation of how much change happened in the update (i.e. information gain or loss)
  * **Want this value to SMOOTHLY DECREASE during training and be as close as possible to 0**
  * **Should be DECREASING**
- ``explained_variance``: Fraction of the return variance explained by the value function. This metric calculates how good the value function is as a predicator of future rewards
  * **Want this value to be as close as possible to 1 (i.e.perfect predictions) during training rather than less than or equal to 0 (i.e. no predictive power)**
  * **Should be INCREASING**
- ``loss``: called total loss is the the overall loss function
  * **Want to MINIMIZE this during training** 
  * **Should be DECREASING**
- ``value_loss``: error that value function is incurring 
  *   **Want to MINIMIZE this during training to 0 (though as discussed this isn't always possible due to randomness)**
  * **Should be DECREASING**





In [ ]:
model.learn(total_timesteps=MAX_STEPS, callback=[eval_callback,TensorboardCallback()])

# Prediction and Printout of Agent's Trading Strategy on Test Data

In [ ]:
model = PPO.load(f"/content/drive/MyDrive/RLmodels/bestPPO/{REWARD}.zip")
env = StockTradingEnv(test, random_ofs_on_reset=False, reward_cal=ratio[0])
obs = env.reset()
for i in range(len(test.date)):
  action, _states = model.predict(obs, deterministic=False)
  obs, rewards, done, info = env.step(action)
  env.render()
  if done:
    break

# TensorBoard Analysis

In [ ]:
%tensorflow_version 2
%load_ext tensorboard
%tensorboard --logdir /content/PPO_SPY_tensorboard/ 